In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


In [2]:
import matplotlib.pyplot as plt


def plot_graphs(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric], '')
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend([metric, 'val_'+metric])

In [3]:
dataset_dir = '..\\data'
train_dir = f"{dataset_dir}\\train"
test_dir = f"{dataset_dir}\\test"
# train_dir = f"{dataset_dir}\\tr"
# test_dir = f"{dataset_dir}\\t"
batch_size = 32
seed = 42

In [4]:
train_dataset = preprocessing.text_dataset_from_directory(
    train_dir,
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed)

test_dataset = preprocessing.text_dataset_from_directory(
    test_dir, batch_size=batch_size)

Found 13567 files belonging to 3 classes.
Using 10854 files for training.
Found 13433 files belonging to 3 classes.


In [5]:
for text_batch, label_batch in train_dataset.take(1):
    for i in range(10):
        print("Sentence: ", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])
example = train_dataset.take(1)

Sentence:  b'S.V.S tuumaat ajatuksesta.??'
Label: 1
Sentence:  b'Multa ainaskii mukavia unijukan kuvia kaikille:)))'
Label: 2
Sentence:  b'Jauhoista ei puhuttu mit\xc3\xa4\xc3\xa4n, mutta se nyt oli taas sivuseikka.........'
Label: 1
Sentence:  b'suht tuore leffa..'
Label: 1
Sentence:  b'Kovin on suppea sanavarastosikin; "sommiporo", "Buahhahhaahaaa" jne.'
Label: 0
Sentence:  b'Toinen kaste, toinen henki'
Label: 1
Sentence:  b'Siihen asti, tutustukaa paikkoihin ja nauttikaa olostanne!'
Label: 2
Sentence:  b'Ja voiko n\xc3\xa4pp\xc3\xa4imist\xc3\xb6\xc3\xa4 k\xc3\xa4ytt\xc3\xa4\xc3\xa4 pelien pelaamiseen?'
Label: 1
Sentence:  b'Suosittelen Artiklan kursseja l\xc3\xa4mpim\xc3\xa4sti! :)'
Label: 2
Sentence:  b'Kiitos viel\xc3\xa4 ehdin muuttaa tiettyyn hotelliin..'
Label: 2


In [6]:
for text_batch, label_batch in train_dataset.take(1):
    print("Sentence: ", text_batch.numpy()[:3])
    print("Label:", label_batch.numpy()[:3])

Sentence:  [b'JA huom!'
 b'Ilmiantokulttuuri kaikessa vastenmielisyydess\xc3\xa4\xc3\xa4n on t\xc3\xb6rke\xc3\xa4t\xc3\xa4 riippumatta aiheesta.'
 b'Tavallisesti k\xc3\xa4ytetyiss\xc3\xa4 valaistuksissa, alle 300\xe2\x80\x93500 lx, l\xc3\xa4mpim\xc3\xa4t s\xc3\xa4vyt koetaan luonteviksi.']
Label: [1 0 1]


In [7]:
for i, label in enumerate(train_dataset.class_names):
    print("Label", i, "corresponds to", label)

Label 0 corresponds to neg
Label 1 corresponds to neut
Label 2 corresponds to pos


In [8]:
VOCAB_SIZE = 1000
encoder = TextVectorization(max_tokens=VOCAB_SIZE)
encoder.adapt(train_dataset.map(lambda text, label: text))
for text_batch, label_batch in train_dataset.take(1):
    for i in range(2):
        print("Sentence: ", text_batch.numpy()[i])
        print("Label:", label_batch.numpy()[i])

Sentence:  b'Onko kukaan lukenut sit\xc3\xa4 juttua?'
Label: 1
Sentence:  b'Hiano mies varmana k\xc3\xa4yn keikalla jatkossakin!!'
Label: 2


In [9]:
vocab = np.array(encoder.get_vocabulary())
print(vocab[:20])

['' '[UNK]' 'ja' 'on' 'ei' 'että' 'se' 'kun' 'niin' 'mutta' 'jos' 'ole'
 'en' 'tai' 'olen' 'voi' 'kuin' 'nyt' 'oli' 'sen']


In [10]:
for text_batch, label_batch in train_dataset.take(1):
    encoded_example = encoder(text_batch)[:3].numpy()
    print(encoded_example)
    for n in range(3):
        print("Original: ", text_batch[n].numpy())
        print("Round-trip: ", " ".join(vocab[encoded_example[n]]))
        print()

[[  6  17   4   1  11   1   1   1  39   3   1   0   0   0   0   0   0   0
    0   0   0]
 [ 84   1   1   1   1  57   1   1   0   0   0   0   0   0   0   0   0   0
    0   0   0]
 [447   1   1   1   1 293  12   1   1  99   1   1  57 560 228   1   0   0
    0   0   0]]
Original:  b'Se nyt ei tietenk\xc3\xa4\xc3\xa4n ole rikos - kukapa meist\xc3\xa4 aina on herttainen?'
Round-trip:  se nyt ei [UNK] ole [UNK] [UNK] [UNK] aina on [UNK]          

Original:  b'M\xc3\xa4 tulin pelonsekaisin tuntein katsoon, onko novellini poistettu.'
Round-trip:  mä [UNK] [UNK] [UNK] [UNK] onko [UNK] [UNK]             

Original:  b'-Meille kaapelitaloudessa asuville laitetaan maksuja, joita en aio taatusti ilman taistelua maksaa..Hmm. Onko kokemuksia t\xc3\xa4st\xc3\xa4 sis\xc3\xa4antennista?'
Round-trip:  meille [UNK] [UNK] [UNK] [UNK] joita en [UNK] [UNK] ilman [UNK] [UNK] onko kokemuksia tästä [UNK]     



In [11]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3)
])
print([layer.supports_masking for layer in model.layers])

[False, True, True, True, True, True, True]


In [12]:
sample_text = """
Kyllä nelivetoinen on huonolla kelillä vakaampi ajaa kuin kaksivetoinen. 
on helvetin hyvä ajettava niin kuivalla kuin liukkallakin kelillä. Saattaahan neliveto olla parempi, mutta takavetosella on pärjätty, ja pärjätään, tosi hyvin. Itse en kaipaa nelivetoa, mutta jos joku tuntee olonsa turvallisemmaksi niin siitä vaan! 
mitähän mahdat tarkoittaa tolla vakaampi ajaa?? suurin merkitys on kyllä itse autolla vrt. paska/keskinkertainen neliveto vs hyvä etuveto. Ei esim audin nelivedot mitenkään ajettavuudella loista muiden yläpuolella. 
"""
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

[-0.01384239  0.01422525 -0.00378018]


In [13]:
padding = "the " * 2000
predictions = model.predict(np.array([sample_text, padding]))
print(predictions[0])

[-0.01384239  0.01422524 -0.00378018]


In [14]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_dataset, epochs=50,
                    validation_data=test_dataset,
                    validation_steps=30)

Epoch 1/50
340/340 [==============================] - 32s 61ms/step - loss: 0.8702 - accuracy: 0.7298 - val_loss: 0.7655 - val_accuracy: 0.7427
Epoch 2/50
340/340 [==============================] - 15s 43ms/step - loss: 0.7817 - accuracy: 0.7314 - val_loss: 0.7374 - val_accuracy: 0.7417
Epoch 3/50
340/340 [==============================] - 16s 46ms/step - loss: 0.7585 - accuracy: 0.7318 - val_loss: 0.7149 - val_accuracy: 0.7510
Epoch 4/50
177/340 [==============>...............] - ETA: 7s - loss: 0.7383 - accuracy: 0.7378

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset)

print('Test Loss:', test_loss)
print('Test Accuracy:', test_acc)

In [ ]:
plt.figure(figsize=(16, 6))
plt.subplot(1, 2, 1)
plot_graphs(history, 'accuracy')
plt.subplot(1, 2, 2)
plot_graphs(history, 'loss')